In [2]:
import os
import string
import re
from itertools import zip_longest

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize, wordpunct_tokenize
punkt = string.punctuation+'»«–…'

from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from judicial_splitter import splitter

from tqdm import tqdm_notebook as tqdm

import json
import pickle
from scipy.sparse import csr_matrix, load_npz

from heapdict import heapdict
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
import math
from collections import defaultdict
from itertools import islice

/home/dkbrz/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
data = pd.read_csv('./data/data.csv', sep='\t', keep_default_na=False)

In [4]:
data.head()

,address,name,title,График работы,Опыт работы,Сфера деятельности,text
0,"Москва, м. Бульвар Дмитрия Донского, Старокача...",menedzher_-konsultant_951375511,Менеджер -консультант,полный день,не имеет значения,Продажи,Компании Мастер-трон требуется офис менеджер....
1,,ohrannik_v_tts_vodnyy_1201120983,Охранник в ТЦ Водный,сменный график,не имеет значения,"Охрана, безопасность",
2,"Москва, м. Дмитровская, Складочная улица, 1с13",uborka_ofisa_1535259521,Уборка офиса,,,,Требуется сотрудник для уборки офисного помеще...
3,"м. Охотный ряд, \n Москва",naborschik_zakazov_1502732417,Наборщик заказов,сменный график,более 1 года,"Транспорт, логистика",
4,"Москва, м. Нахимовский проспект, Нахимовский п...",voditel_taksi_1394522421,Водитель такси,сменный график,более 3 лет,Автомобильный бизнес,


In [ ]:
data['text'] = [open('./corpora/{}'.format(i)).read() for i in data.name]

In [7]:
data['text'] = [open('./corpora/{}'.format(i)).read() for i in data.name]

In [5]:
data2 = data

In [7]:
data2.columns = ['Адрес', 'name', 'Заголовок', 'График работы', 'Опыт работы',
       'Сфера деятельности', 'Объявление']

In [10]:
del data2['name']

In [11]:
del data2['Адрес']

In [13]:
data2.columns

Index(['Заголовок', 'График работы', 'Опыт работы', 'Сфера деятельности',
       'Объявление'],
      dtype='object')

In [14]:
data2 = data2[['Заголовок','Объявление','График работы','Опыт работы','Сфера деятельности']]

In [15]:
data2.to_csv('./data/data2.csv', sep='\t', index=False)

In [16]:
data.to_csv('./data/data.csv', sep='\t', index=False)

# Preprocessing

In [41]:
def preprocessing(text, stop=False):
    global wordpunct_tokenize, stop_words, punkt
    #text = re.sub(r"([a-zа-я0-9])(.)([A-ZА-Я0-9])", r"\1\2 \3", text)
    text = word_tokenize(text)
    new_text= []
    for word in text:
        word = word.strip(punkt)
        if word:
            if word in punkt: continue
            elif word.isdigit(): continue
            elif stop and word in stop_words: continue
            else: new_text.append(morph.parse(word)[0].normal_form)
    return new_text

In [4]:
titles_data = defaultdict(list)
def write():
    n = 0
    with open('d2v_answers_3.txt','w', encoding = 'utf-8') as answers:
        with open ('d2v_indexes.txt', 'w', encoding = 'utf-8') as indexes:
                for value in tqdm(data['name']):
                    with open ('./corpora/{}'.format(value), 'r') as f:
                        article = f.read()
                    for chunk in splitter(article, 2):
                        answer = ' '.join(preprocessing(chunk))
                        if len(answer) > 5:
                            answers.write(answer+'\n')
                            titles_data[value].append(n)
                            n+=1
                            indexes.write(value+'\n')
    with open('titles_data.json','w') as f:
        json.dump(dict(titles_data), f, ensure_ascii=False)
                            
write()                          

In [5]:
class LabeledLineSentence(object):
    def __init__(self, name):
        self.name = name
    def __iter__(self):
        for key, value in enumerate(zip_longest(open('d2v_indexes.txt','r'), open('d2v_answers_3.txt','r'))):
            yield TaggedDocument(words=value[1].strip().split(), tags=[value[0].strip()])

In [6]:
d2v_model = Doc2Vec(vector_size=100, min_count=2, alpha=0.025, seed = 23,
                min_alpha=0.025, epochs=100, workers=8, dm=1)

%time d2v_model.build_vocab(LabeledLineSentence(''))
print (len(d2v_model.wv.vocab))
%time d2v_model.train(LabeledLineSentence(''), total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

CPU times: user 421 ms, sys: 8.05 ms, total: 429 ms
Wall time: 423 ms
9714
CPU times: user 2min 54s, sys: 25.4 s, total: 3min 20s
Wall time: 1min 48s


In [7]:
d2v_model.save('d2v_1000')

In [10]:
d2v_data = []
def save_d2v_base():
    for line in tqdm(open('d2v_answers_3.txt','r')):
            d2v_model.random.seed(23)
            d2v_data.append(d2v_model.infer_vector(line.strip().split()).tolist())
    with open('d2v_data.json','w') as f:
        json.dump(d2v_data, f, ensure_ascii=False)
%time save_d2v_base()


CPU times: user 1min 2s, sys: 0 ns, total: 1min 2s
Wall time: 1min 1s


In [12]:
def write():
    global qa_corpus
    with open('ok_answers.txt','w', encoding = 'utf-8') as answers:
                for value in tqdm(data['name']):
                    with open ('./corpora/{}'.format(value), 'r') as f:
                        article = f.read()
                    answer = ' '.join(preprocessing(article))
                    answers.write(answer+'\n')
write()

In [4]:
tfidf = TfidfVectorizer(ngram_range = (1,1), stop_words=stop_words)
with open ('ok_answers.txt','r') as f:
    corpus = f.readlines()
tfidf.fit(corpus)
with open ('tfidf', 'wb') as output:
    pickle.dump(tfidf, output)

In [3]:
def get_w2v_vectors(text, k=300, prep=True):
    """Получает вектор документа"""
    
    global w2v_model, stop_words, word_tokenize, tfidf
    
    if prep:
        arr_text = preprocessing(text, stop=True)
    else:
        arr_text = text.split()
    n = 0
    vector = np.array([0]*300)
    address = {key:value for key, value in enumerate(tfidf.transform([' '.join(arr_text)]).toarray()[0]) if value != 0}
    for word in set(arr_text):
        if word not in stop_words:
            try:
                weight = address[tfidf.vocabulary_[word]]
                vec = np.array(w2v_model.wv[word]*weight)
                vector = vector + vec
                n += weight
            except:
                continue
    if n > 0: vector = vector / n
    
    return vector

In [15]:
w2v_data = []
def save_w2v_base():
    for line in tqdm(open('d2v_answers_3.txt','r')):
            w2v_data.append(get_w2v_vectors(line.strip(), k=300, prep=False).tolist())
    with open('w2v_data.json','w') as f:
        json.dump(w2v_data, f, ensure_ascii=False)
%time save_w2v_base()


CPU times: user 1min 3s, sys: 0 ns, total: 1min 3s
Wall time: 1min 2s


In [84]:
def save_json_title():
    w2v_titles = []
    for item in tqdm(data['title'].values):
        w2v_titles.append(get_w2v_vectors(item.strip(), k=300, prep=True).tolist())
    with open('w2v_titles.json', 'w') as outfile:
        json.dump(w2v_titles, outfile, ensure_ascii=False)
save_json_title()

In [21]:
def save_json_area():
    area = []
    for item in tqdm(data['Сфера деятельности'].values):
        area.append(get_w2v_vectors(item.strip(), k=300, prep=True).tolist())
    with open('area.json', 'w') as outfile:
        json.dump(area, outfile, ensure_ascii=False)
save_json_area()

In [4]:
from collections import Counter
import math
from collections import defaultdict
from itertools import islice
import numpy as np

def get_term_doc_matrix():
    n = 9947
    dictionary = {}
    term_doc_matrix = []
    for key, item in enumerate(open('ok_answers.txt','r')):
        text = Counter(item.strip().split())
        for word in text:
            if word in dictionary:
                term_doc_matrix[dictionary[word]][key] += text[word]
            else:
                dictionary[word] = len(dictionary)
                term_doc_matrix.append(np.zeros(n))
                term_doc_matrix[dictionary[word]][key] += text[word]
    return dictionary, term_doc_matrix

def inverted_index(dictionary, term_doc_matrix) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    result = {}
    for word in dictionary:
        result[word] = {key: int(value) for key, value in enumerate(term_doc_matrix[dictionary[word]]) if value > 0}
    return result

In [49]:
def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = math.log(1 + (N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

def compute_sim(word) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    global index, dictionary, term_doc_matrix, doc_length, avgdl, N
    if word in dictionary:
        n = len(index[word])
        result = {}
        idx = dictionary[word]
        for doc in index[word]:
            qf = term_doc_matrix[idx,int(doc)]
            score = score_BM25(qf, doc_length[doc], avgdl, k1, b, N, n)
            result[int(doc)] = score
        return result
    else:
        return {}

def get_okapi(query) -> float:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    query = query.split()
    result = [0]*9947
    for word in query:
        current = compute_sim(word)
        for doc in current:
            result[doc] += current[doc]
    return result

In [139]:
def score_BM25(qf,n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    global index, dictionary, term_doc_matrix, doc_length, avgdl, N, k1, b, blength
    score = math.log(1 + (N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+blength)
    return score

def get_okapi(query) -> float:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    global index, dictionary, term_doc_matrix, doc_length, avgdl, N, k1, b
    query = query.split()
    result = csr_matrix((1,9947), dtype=int)
    for word in query:
        if word in dictionary:
            idx = dictionary[word]
            n = (term_doc_matrix[idx]>0).sum()
            result += score_BM25(term_doc_matrix[idx].todense(),n)
    return np.nan_to_num(np.array(result))

In [24]:
(term_doc_matrix[4]>0).sum()

216

In [14]:
csr_matrix([[1,2],[3,4]]).todense()+1

matrix([[2, 3],
        [4, 5]], dtype=int64)

In [36]:
get_okapi('менеджер')

216


array([[0.12865694, 0.        , 0.        , ..., 0.        , 0.        ,
        0.00935441]])

In [51]:
get_okapi('менеджер')

3372


matrix([[0.03636032,        nan, 0.        , ..., 0.        , 0.        ,
         0.00264369]])

In [71]:
term_doc_matrix[4].nonzero()

(array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 array([   0,    1,    3, ..., 9938, 9942, 9946], dtype=int32))

In [65]:
for i in term_doc_matrix[4].nonzero()[0]:
    print (i)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [40]:
dictionary['менеджер']

4

In [63]:
for i in term_doc_matrix[4]:
        print (i)

  (0, 0)	0.021739130434782608
  (0, 1)	nan
  (0, 3)	nan
  (0, 4)	nan
  (0, 5)	nan
  (0, 7)	nan
  (0, 8)	nan
  (0, 15)	nan
  (0, 17)	nan
  (0, 18)	nan
  (0, 22)	nan
  (0, 23)	nan
  (0, 32)	nan
  (0, 33)	nan
  (0, 34)	nan
  (0, 36)	nan
  (0, 38)	nan
  (0, 41)	nan
  (0, 43)	nan
  (0, 48)	nan
  (0, 51)	nan
  (0, 52)	nan
  (0, 54)	nan
  (0, 57)	nan
  (0, 60)	nan
  :	:
  (0, 9850)	nan
  (0, 9851)	nan
  (0, 9852)	nan
  (0, 9855)	nan
  (0, 9862)	nan
  (0, 9864)	nan
  (0, 9871)	nan
  (0, 9873)	nan
  (0, 9874)	nan
  (0, 9875)	nan
  (0, 9879)	nan
  (0, 9884)	nan
  (0, 9896)	nan
  (0, 9903)	nan
  (0, 9907)	0.008547008547008548
  (0, 9910)	nan
  (0, 9912)	nan
  (0, 9916)	nan
  (0, 9917)	nan
  (0, 9930)	nan
  (0, 9931)	nan
  (0, 9933)	nan
  (0, 9938)	nan
  (0, 9942)	nan
  (0, 9946)	0.003424657534246575


In [7]:
dictionary, term_doc_matrix = get_term_doc_matrix()
index = inverted_index(dictionary, term_doc_matrix)

k1 = 2.0
b = 0.75

doc_length = {}
for key, value in enumerate(np.transpose(term_doc_matrix)):
    doc_length[key] = sum(value)

avgdl = sum(doc_length.values())/len(doc_length)
N = len(doc_length)
print (avgdl, N)

48.64381220468483 9947


In [10]:
i = [doc_length[i] for i in sorted(doc_length)]

In [11]:
copy = np.array(term_doc_matrix)/np.array([i])

In [12]:
del term_doc_matrix

In [13]:
from scipy.sparse import csr_matrix, save_npz

In [14]:
with open('index.json', 'w') as outfile:
        json.dump(index, outfile, ensure_ascii=False)
with open('dictionary.json', 'w') as outfile:
        json.dump(dictionary, outfile, ensure_ascii=False)

with open('doc_length.json', 'w') as outfile:
        json.dump(doc_length, outfile, ensure_ascii=False)
        
term_doc_matrix = csr_matrix(copy)
print (term_doc_matrix.nnz)
term_doc_matrix.eliminate_zeros()
print (term_doc_matrix.nnz)
del copy
save_npz('term_doc_matrix.npz', term_doc_matrix)

59507472
59507472


In [20]:
term_doc_matrix[0]=0

# Work

In [7]:
def cos(a,b):
    lnorm = np.sqrt(np.sum(a**2))
    rnorm = np.sqrt(np.sum(b**2))
    return np.dot(a, b)/(lnorm*rnorm)

In [5]:
w2v_model = KeyedVectors.load('w2v_model')
d2v_model = Doc2Vec.load('d2v_1000')
term_doc_matrix = load_npz('term_doc_matrix.npz')

In [6]:
with open ('tfidf', 'rb') as output:
    tfidf = pickle.load(output)
    
with open('index.json', 'r') as outfile:
    index = json.load(outfile)

with open('dictionary.json', 'r') as outfile:
    dictionary = json.load(outfile)

with open('doc_length.json', 'r') as outfile:
    doc_length = json.load(outfile)

In [8]:
k1 = 2.0
b = 0.75
avgdl = 48.64381220468483
N = 9947

In [9]:
doc_length = [doc_length[i]/avgdl for i in sorted(doc_length)]

In [102]:
blength = k1*(1-b+b*np.array(doc_length))

In [37]:
with open('w2v_titles.json','r') as f:
    w2v_titles = np.array(json.load(f))

with open('titles_data.json','r') as f:
    titles_data = json.load(f)
    
with open('d2v_data.json','r') as f:
    d2v_data = np.array(json.load(f))
    
with open('area.json','r') as f:
    area = np.array(json.load(f))
    
with open('w2v_data.json','r') as f:
    w2v_data = np.array(json.load(f))

In [178]:
def _search(query, n=10):
    query = ' '.join(preprocessing(query))
    
    q_w2v = np.array(get_w2v_vectors(query, k=300, prep=False))
    d2v_model.random.seed(23)
    q_d2v = np.array(d2v_model.infer_vector(query.strip().split()))
    
    okapi = np.array(get_okapi(query))
    
    w_d = cosine_similarity([q_w2v], w2v_data)+cosine_similarity([q_d2v], d2v_data)
    w_d = np.array([max(w_d[0][titles_data[title]]) if title in titles_data else 0 for title in data['name'].values])
    w_d = sigmoid(w_d * (1+ np.log(1+okapi)))
    
    result = 0.5*w_d + 0.3*cosine_similarity([q_w2v], w2v_titles) + 0.2*cosine_similarity([q_w2v], area)
    
    return result

In [57]:
%load_ext line_profiler

In [55]:
query = 'администратор в отель'

In [167]:
%lprun -f _search _search(query)

In [140]:
%lprun -f get_okapi get_okapi('менеджер')

In [141]:
%lprun -f score_BM25 score_BM25(term_doc_matrix[4].todense(), 216)

In [142]:
csr_matrix((1,9947), dtype=int).todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [45]:
%timeit _search('администратор в отель', n=10)

332 ms ± 4.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [89]:
sum((term_doc_matrix[4]>0)[0])

<1x9947 sparse matrix of type '<class 'numpy.bool_'>'
	with 216 stored elements in Compressed Sparse Row format>

In [143]:
%timeit _search('администратор в отель', n=10)

165 ms ± 573 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [205]:
def search(query, n=10):
    result = _search(query, n=10)
    result = pd.Series(data = result[0],
                       index = data['name'].values).sort_values(ascending=False)[:n]
    result = [open('./corpora/{}'.format(i)).read() for i in result.index]
    return result
    #for key, i in enumerate(result.index):
    #    print ('----------------------------------------------------')
    #    print (result[key], '\n', open('./corpora/{}'.format(i)).read())

In [206]:
search('администратор в отель')

['Описание работодателя: Мини-Отель приглашает на работу Администратора.\nТребования: Грамотная устная и письменная речь, коммуникабельность, энергичность, работоспособность, ответственность, внимательность, доброжелательность, стрессоустойчивость, умение работать в режиме многозадачности. Умение работать в команде на общий результат.\nОбязанности: Грамотная устная и письменная речь, коммуникабельность, энергичность, работоспособность, ответственность, внимательность, доброжелательность, стрессоустойчивость, умение работать в режиме многозадачности. Умение работать в команде на общий результат.\nУсловия: Вахтовый метод, 15/15 или 20/20.',
 'Требуется администратор c опытом работы в отель. з.п. от выручки\n\nЗнание ПК, стрессоустойчивость, коммуникабельность, ответственность, внимательность, умение разрешать конфликтные ситуации.\nОбязанности: Оформление и заселение гостей, контроль за соблюдением правил проживания в отеле.\nУсловия: График работы сменный.\n\nПодробности на собеседовани

In [198]:
data['text'] = [open('./corpora/{}'.format(name), 'r').read() for name in data['name']]

In [ ]:
def search(query, n=10):
    result = _search(query, n=10)
    df = pd.DataFrame['']

In [186]:
result = _search(query, n=10)

In [195]:
del data['result']

In [201]:
data.head(30)

,address,name,title,График работы,Опыт работы,Сфера деятельности,text
0,"Москва, м. Бульвар Дмитрия Донского, Старокача...",menedzher_-konsultant_951375511,Менеджер -консультант,полный день,не имеет значения,Продажи,Компании Мастер-трон требуется офис менеджер....
1,,ohrannik_v_tts_vodnyy_1201120983,Охранник в ТЦ Водный,сменный график,не имеет значения,"Охрана, безопасность",
2,"Москва, м. Дмитровская, Складочная улица, 1с13",uborka_ofisa_1535259521,Уборка офиса,,,,Требуется сотрудник для уборки офисного помеще...
3,"м. Охотный ряд, \n Москва",naborschik_zakazov_1502732417,Наборщик заказов,сменный график,более 1 года,"Транспорт, логистика",
4,"Москва, м. Нахимовский проспект, Нахимовский п...",voditel_taksi_1394522421,Водитель такси,сменный график,более 3 лет,Автомобильный бизнес,
5,,ohrannik_na_novyy_obekt_1172702846,Охранник на новый объект,вахтовый метод,не имеет значения,"Охрана, безопасность",
6,"Москва, м. Электрозаводская, Большая Почтовая ...",avtoelektrik-diagnost_1581735794,Автоэлектрик-диагност,полный день,более 1 года,Автомобильный бизнес,Обязанности:\n\nВыполнение работ по диагностик...
7,"м. Площадь революции, \n Москва",rabota_vahtoy_upakovschik_tsa_ikry_vahtapitani...,Работа вахтой упаковщик (ца) икры вахта/питание,вахтовый метод,не имеет значения,"Без опыта, студенты",
8,"м. Братиславская, \n ул. Братиславская д. 12",prodavets_yuvelirnyh_izdeliy_metro_bratislavsk...,Продавец ювелирных изделий (метро Братиславская),полный день,не имеет значения,Продажи,
9,"Москва, м. Новогиреево, природно-исторический ...",trebuetsya_voditel_s_kategoriey_s_1678360959,Требуется водитель с категорией С,,,,Описание работодателя: Транспортная компания с...


In [45]:
def sigmoid(x):
      return 1/(1+np.exp(-x))

In [185]:
search('администратор в отель')

----------------------------------------------------
0.6697165153622509 
 Описание работодателя: Мини-Отель приглашает на работу Администратора.
Требования: Грамотная устная и письменная речь, коммуникабельность, энергичность, работоспособность, ответственность, внимательность, доброжелательность, стрессоустойчивость, умение работать в режиме многозадачности. Умение работать в команде на общий результат.
Обязанности: Грамотная устная и письменная речь, коммуникабельность, энергичность, работоспособность, ответственность, внимательность, доброжелательность, стрессоустойчивость, умение работать в режиме многозадачности. Умение работать в команде на общий результат.
Условия: Вахтовый метод, 15/15 или 20/20.
----------------------------------------------------
0.6419669356477771 
 Требуется администратор c опытом работы в отель. з.п. от выручки

Знание ПК, стрессоустойчивость, коммуникабельность, ответственность, внимательность, умение разрешать конфликтные ситуации.
Обязанности: Оформлени

In [265]:
search('администратор в отель')

----------------------------------------------------
0.7632302066243506 
 Требуется администратор c опытом работы в отель. з.п. от выручки

Знание ПК, стрессоустойчивость, коммуникабельность, ответственность, внимательность, умение разрешать конфликтные ситуации.
Обязанности: Оформление и заселение гостей, контроль за соблюдением правил проживания в отеле.
Условия: График работы сменный.

Подробности на собеседовании
----------------------------------------------------
0.7161505684002567 
 Описание работодателя: Мини-Отель приглашает на работу Администратора.
Требования: Грамотная устная и письменная речь, коммуникабельность, энергичность, работоспособность, ответственность, внимательность, доброжелательность, стрессоустойчивость, умение работать в режиме многозадачности. Умение работать в команде на общий результат.
Обязанности: Грамотная устная и письменная речь, коммуникабельность, энергичность, работоспособность, ответственность, внимательность, доброжелательность, стрессоустойчиво

In [274]:
%time search('хочу работать слесарем',n=5)

----------------------------------------------------
0.45082956056503254 
 Автослесарь-универсал требуется в автосервис Любермакс
(автосервис обслуживает легковые автомобили), 
хорошие условия труда, стабильные клиенты.

.
Требования:
Опыт работы слесарем не менее одного года
.
Условия:
График работы сменный, оформление по ТК, 
предоставляется спецодежда. Теплое помещение
.
Хорошие условия!
.

Обязанности:
Послегарантийное ТО (замена масла, колодок, фильтров, 
жидкостей и т.д.), ремонт ходовой.
----------------------------------------------------
0.43994409403861395 
 В автосервис в Северном Бутово требуется автослесарь-шиномонтажник с опытом работы не менее 3 лет. Требования: автослесарные работы по ходовой части автомобиля, пунктуальность, вежливость, умение грамотно общаться с клиентами. График работы сменный, оплата сдельная. Автосервис находится на территории престижного жилого комплекса. Есть комната отдыха, туалет. Слаженный коллектив. Остальные подробности  по телефону.
-------

In [159]:
list(islice(enumerate([1,2,3]), 2))

[(0, 1), (1, 2)]

In [202]:
a = pd.Series(data=[1,2,3], index = ['1','2','3'])

In [210]:
a.sort_values()[:2]

1    1
2    2
dtype: int64

In [148]:
query = ' '.join(preprocessing(query))
    
q_w2v = np.array(get_w2v_vectors(query, k=300, prep=False))
d2v_model.random.seed(23)
q_d2v = np.array(d2v_model.infer_vector(query.strip().split()))

In [220]:
result

0.7008210467337095

In [92]:
sum(w[0])

2643.9732010185253

In [91]:
a = w[0]*o[0]
sum(a)

2963.968987087507

In [34]:
okapi = get_okapi('менеджер')
okapi = np.array([okapi[i] if i in okapi else 0 for i in range(9947)])

In [36]:
1+ np.log(1+okapi)

array([1.12102838, 1.        , 1.        , ..., 1.        , 1.        ,
       1.00412751])

In [33]:
np.log([1])

array([0.])

In [10]:
len(tfidf.vocabulary_)

16088

In [29]:
q_w2v = get_w2v_vectors('окно')

In [49]:
%time cosine_similarity([q_w2v], w2v_titles)+cosine_similarity([q_w2v], area)

CPU times: user 99.9 ms, sys: 50.8 ms, total: 151 ms
Wall time: 40.5 ms


array([[-0.06793828, -0.02098418,  0.31006221, ...,  0.23572483,
         0.04102533,  0.02931117]])

In [50]:
%time cosine_similarity([q_w2v], w2v_titles)+cosine_similarity([q_w2v], area)

CPU times: user 86.9 ms, sys: 77.1 ms, total: 164 ms
Wall time: 41.4 ms


array([[-0.06793828, -0.02098418,  0.31006221, ...,  0.23572483,
         0.04102533,  0.02931117]])

In [52]:
d2v_model.random.seed(23)
q_d2v = np.array([d2v_model.infer_vector('менеджер'.strip().split())])

In [54]:
%time cosine_similarity([q_w2v], w2v_data) + cosine_similarity(q_d2v, d2v_data)

CPU times: user 99 ms, sys: 93.6 ms, total: 193 ms
Wall time: 65.9 ms


array([[0.35750367, 0.52390386, 0.30380934, ..., 0.27620902, 0.36839771,
        0.26782267]])

In [60]:
a = cosine_similarity([q_w2v], w2v_data) + cosine_similarity(q_d2v, d2v_data)

In [67]:
a[0][[1,2,3]]

array([0.52390386, 0.30380934, 0.51364224])

In [71]:
    data['name'].values[0]

'menedzher_-konsultant_951375511'

In [76]:
titles_data['menedzher_-konsultant_951375511']

[0, 1, 2, 3]

In [78]:
items = [max(a[0][titles_data[title]]) if title in titles_data else 0 for title in data['name'].values]

In [83]:
other = cosine_similarity([q_w2v], w2v_titles)+cosine_similarity([q_w2v], area)

In [84]:
%time np.array(items)*2 + other

CPU times: user 5.17 ms, sys: 563 µs, total: 5.73 ms
Wall time: 4.48 ms


array([[ 0.97986944, -0.02098418,  1.50007496, ...,  1.70042668,
         0.94927754,  0.77770367]])

In [30]:
w2v_titles.shape

(9947, 300)

In [8]:
titles = [i['id'] for i in data_json]

In [13]:
arr = np.array([1,2,3,4,5])
arr[[1,2]]

array([2, 3])

In [9]:
titles_data = defaultdict(list)
for key, i in enumerate(titles):
    titles_data[i].append(key)

In [10]:
titles_data = dict(titles_data)

In [11]:
with open('titles_data.json','w') as f:
    json.dump(titles_data, f, ensure_ascii=False)

In [ ]:
data_json = []

In [12]:
w2v_data = [i['w2v'] for i in data_json]
with open('w2v_data.json','w') as f:
    json.dump(w2v_data, f, ensure_ascii=False)

In [ ]:
d2v_data = [i['d2v'] for i in data_json]
with open('d2v_data.json','w') as f:
    json.dump(d2v_data, f, ensure_ascii=False)

In [43]:
with open('data.json', 'r') as outfile:
    data_json = json.load(outfile)

In [44]:
d2v_data = [i['d2v'] for i in data_json]
with open('d2v_data.json','w') as f:
    json.dump(d2v_data, f, ensure_ascii=False)

In [13]:
w2v_titles = [title_json[name] for name in data['name']]
with open('w2v_titles.json','w') as f:
    json.dump(w2v_titles, f, ensure_ascii=False)

In [14]:
w2v_titles = [title_json[name] for name in data['name']]
with open('w2v_titles.json','w') as f:
    json.dump(w2v_titles, f, ensure_ascii=False)

In [30]:
with open('area.json','r') as f:
    area = json.load(f)

In [51]:
q_w2v = get_w2v_vectors('дом')

In [49]:
np.array(w2v_data).shape

(17354, 300)

In [52]:
q_w2v

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [41]:
%time cosine_similarity([q_w2v], np.array(w2v_data))

CPU times: user 168 ms, sys: 46.7 ms, total: 214 ms
Wall time: 163 ms


array([[0., 0., 0., ..., 0., 0., 0.]])

In [22]:
np.array([1,2])*np.array([1,2])

array([1, 4])

In [43]:
%timeit np.zeros(300)

605 ns ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [44]:
%timeit np.array([0]*300)

11.6 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [30]:
a = csr_matrix((1,9947), dtype=int)

In [39]:
type(a) == csr_matrix

True

In [38]:
csr_matrix

scipy.sparse.csr.csr_matrix

In [29]:
np.array([9]*9927) + 1+ np.log(1+a.toarray()[0])

ValueError: operands could not be broadcast together with shapes (9927,) (9947,) 